# Notebook to calculate homeranges and create final dataframes

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import rasterio
import geopandas as geopd
import rasterio.rio
import seaborn as sns
import datetime as dt 

from rasterio.plot import show
from scipy.spatial import ConvexHull

import pyreadr

import sys
sys.path.append("../modeling")
import home_ranges as hr



First, read in the cleaned data

In [ ]:
foxes_all = geopd.read_file("../data/cleaned_shapefiles/foxes_all.shp")
foxes_resamp = geopd.read_file("../data/cleaned_shapefiles/foxes_resamp.shp")
sample_points = geopd.read_file("../data/cleaned_shapefiles/sample_points.shp")


Build infiv_fox_all, which contains all data for each individual fox

In [ ]:
indiv_fox_all = {}
for i in foxes_all.id.unique():
    indiv_fox_all[i] = foxes_all[foxes_all.id == i]

In [ ]:
foxes = list(indiv_fox_all.values())


Building indiv_fox_resamp, which contains the resampled data for each individual fox

In [ ]:
indiv_fox_resamp = {}
for i in foxes_resamp.id.unique():
    indiv_fox_resamp[i] = foxes_resamp[foxes_resamp.id == i]

In [ ]:
foxes_indiv_resamp = list(indiv_fox_resamp.values())

## __Function Definition for Home Range plotting__
The necessary functions are defined in the python script "home_ranges" in the "modeling" folder.

#### __Plotting the Home Ranges__

For the foxes with all points.

In [ ]:
for i in foxes:
    hr.hr_plot(i, 0.95)

For the foxes with the resampled points

In [ ]:
for i in foxes_indiv_resamp:
    hr.hr_plot(i, 0.95)

## __Final Data Frame for all Foxes__

### Creating the Geometries for the Home Ranges

In [ ]:
fox_area = [hr.hr_geometry_df(fox, 0.95) for fox in foxes]
fox_hulls = [hr.hr_area(fox, 0.95) for fox in foxes]

### Deleting the Points within the Home Ranges from the Sample Points

In [ ]:
sample_points_all = sample_points.copy()
for fox in fox_hulls:
    sample_points_all = sample_points_all.difference(fox)

### Processing the Data Frames for the Sample Points and the Foxes

#### Creating Data Frame for Foxes

The resulting data frame will only include the GPS data points of the foxes, that lie within the home range areas of these foxes. 

In [ ]:
foxes_all = []
for i in range(0,len(foxes)-1):
    if i == 0:
        foxes_all = pd.concat([foxes[i], foxes[i+1]])
    else:
        foxes_all = pd.concat([foxes_all, foxes[i+1]])

In [ ]:
fox_all = []
for i in range(0,len(fox_area)-1):
    if i == 0:
        fox_all = pd.concat([fox_area[i], fox_area[i+1]])
    else:
        fox_all = pd.concat([fox_all, fox_area[i+1]])

In [ ]:
fox_all_merged = foxes_all.merge(fox_all, on = ["geometry", "id"])

#### Creating Data Frame for Sample Points

The resulting data frame will only include the GPS points of the sample points that lie outside of the home range areas of the foxes. 

In [ ]:
sample_points_df = geopd.GeoDataFrame(geometry = sample_points_all)

In [ ]:
sample_points_all_merged = sample_points.merge(sample_points_df, on = "geometry")
sample_points_all_merged.shape

### Combining the Data Frames into Final Data Frame for all Foxes

First the value for the target variable is put into a column in both data frames. Further, we set the id for the sample points to "available", as these are the points that were available to the foxes, but not used in their home ranges. Further we rename "x" and "y" in the sample points to match "x_" and "y_" in the fox data frame.

In [ ]:
fox_all_merged["target"] = 1


sample_points_all_merged["target"] = 0
sample_points_all_merged["id"] = "available"
sample_points_all_merged = sample_points_all_merged.rename(columns = {"x" : "x_",
                                            "y" : "y_"})

Then we concatenate both data frames, resulting in a combined data frame, with NaN values in the target = 0 columns for timestamp, sex and t_, as we do not have meaningful values for available points for these features.

In [ ]:
df_all = pd.concat([fox_all_merged, sample_points_all_merged])

And finally we export the resulting data frame as shapefile for use in future notebooks.

In [ ]:
df_all.to_file("../data/final_shapefiles/foxes_modelling_all.shp")


## __Final Data Frame for resampled Foxes__

### Creating the Geometries for the Home Ranges

In [ ]:
fox_area_resamp = [hr.hr_geometry_df(fox, 0.95) for fox in foxes_indiv_resamp]
fox_hulls_resamp = [hr.hr_area(fox, 0.95) for fox in foxes_indiv_resamp]

### Deleting the Points within the Home Ranges from the Sample Points

In [ ]:
sample_points_resamp = sample_points.copy()
for fox in fox_hulls_resamp:
    sample_points_resamp = sample_points_resamp.difference(fox)

### Processing the Data Frames for the Sample Points and the resampled Foxes

#### Creating Data Frame for Foxes

The resulting data frame will only include the GPS data points of the foxes, that lie within the home range areas of these foxes. 

In [ ]:
foxes_resamp = []
for i in range(0,len(foxes_indiv_resamp)-1):
    if i == 0:
        foxes_resamp = pd.concat([foxes_indiv_resamp[i], foxes_indiv_resamp[i+1]])
    else:
        foxes_resamp = pd.concat([foxes_resamp, foxes_indiv_resamp[i+1]])

In [ ]:
fox_resamp = []
for i in range(0,len(fox_area_resamp)-1):
    if i == 0:
        fox_resamp = pd.concat([fox_area_resamp[i], fox_area_resamp[i+1]])
    else:
        fox_resamp = pd.concat([fox_resamp, fox_area_resamp[i+1]])

In [ ]:
fox_resamp_merged = foxes_resamp.merge(fox_resamp, on = ["geometry", "id"])
fox_resamp_merged.shape

#### Creating Data Frame for Sample Points

The resulting data frame will only include the GPS points of the sample points that lie outside of the home range areas of the foxes. 

In [ ]:
sample_resamp_df = geopd.GeoDataFrame(geometry = sample_points_resamp)

In [ ]:
sample_points_resamp_merged = sample_points.merge(sample_resamp_df, on = "geometry")
sample_points_resamp_merged.shape

In [ ]:
sample_points_resamp_merged = sample_points_resamp_merged.iloc[::5, :]
sample_points_resamp_merged.shape

### Combining the Data Frames into Final Data Frame for resampled Foxes

First the value for the target variable is put into a column in both data frames. Further, we set the id for the sample points to "available", as these are the points that were available to the foxes, but not used in their home ranges. Further we rename "x" and "y" in the sample points to match "x_" and "y_" in the fox data frame.

In [ ]:
fox_resamp_merged["target"] = 1


sample_points_resamp_merged["target"] = 0
sample_points_resamp_merged["id"] = "available"
sample_points_resamp_merged = sample_points_resamp_merged.rename(columns = {"x" : "x_",
                                            "y" : "y_"})

Then we concatenate both data frames, resulting in a combined data frame, with NaN values in the target = 0 columns for timestamp, sex and t_, as we do not have meaningful values for available points for these features.

In [ ]:
df_resamp = pd.concat([fox_resamp_merged, sample_points_resamp_merged])
df_resamp.head()

In [ ]:
df_resamp.shape

And finally we export the resulting data frame as shapefile for use in future notebooks.

In [ ]:
df_resamp.to_file("../data/final_shapefiles/foxes_modelling_resamp.shp")
